In [48]:
# # Reading current directory
# !pwd

# # Mounting directory
# # by clicking mount drive
# from google.colab import drive
# drive.mount('/content/drive')

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
# Importing all libraries and modules
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense

In [50]:
# loading customer churn_Modelling.csv data from drive
customer_data = pd.read_csv('/content/drive/My Drive/Datasets/Churn_Modelling.csv')

In [51]:
customer_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [52]:
# Dropping data that never put any sense for our model
# dropping RowNumber, CustomerId,Surname  from training data
# Creating input data and output labels

input_data = customer_data.drop(labels=['RowNumber','CustomerId','Surname','Exited'],axis =1)
output_label = customer_data['Exited']

In [53]:
input_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [54]:
output_label.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [55]:
# We know that our model only reads numerical data
# We need to map string values to numerical values using either one hot encoder or LabelEncoder
# So, Geography and Gender must be encoded

from sklearn.preprocessing import LabelEncoder
label_geography = LabelEncoder()
input_data['Geography']= label_geography.fit_transform(input_data['Geography'])
input_data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,Female,42,2,0.00,1,1,1,101348.88
1,608,2,Female,41,1,83807.86,1,0,1,112542.58
2,502,0,Female,42,8,159660.80,3,1,0,113931.57
3,699,0,Female,39,1,0.00,2,0,0,93826.63
4,850,2,Female,43,2,125510.82,1,1,1,79084.10


In [56]:
# Three geographical location encoded as 0,1,2
customer_data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [57]:
customer_data['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [58]:
# Encodeing Gender
label_gender = LabelEncoder()
input_data['Gender']= label_gender.fit_transform(input_data['Gender'])
input_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [59]:
# Categorical data must be converted into one hot encoding
# To remove redundancy we removed first, removing one make 
# make same sense to model
input_data = pd.get_dummies(input_data,drop_first=True,columns=['Geography'])
input_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_1,Geography_2
0,619,0,42,2,0.00,1,1,1,101348.88,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,1


In [60]:
# Splitting training and testing data
from sklearn.model_selection import train_test_split

In [61]:
x_train, x_test, y_train, y_test = train_test_split( input_data,output_label, test_size=0.3, random_state=101,stratify =output_label) 

In [62]:
# Feature standardization/normalization( Make sure all features have same scale)

from sklearn.preprocessing import StandardScaler


In [63]:
scaler = StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [64]:
x_train[0]

array([-1.0093902 ,  0.92614765, -0.75456609, -1.37875442, -1.22086358,
        0.79746457,  0.64198477,  0.98187845,  1.47743017, -0.5780101 ,
       -0.57141165])

In [65]:
x_train.shape

(7000, 11)

In [66]:
#x_train.shape
# There are 11 columns
x_train[1].shape

(11,)

In [67]:
# Building Model
model = Sequential()
model.add(Dense(x_train.shape[1],activation='relu',input_dim= x_train.shape[1])) # input layer with 11 node
model.add(Dense(128,activation='relu'))# Hidden layer
model.add(Dense(1,activation='sigmoid'))# Output layer having single node (0/1)

In [68]:
# Compiling model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [83]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_7 (Dense)              (None, 128)               1536      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 1,797
Trainable params: 1,797
Non-trainable params: 0
_________________________________________________________________


In [81]:
# Training model
model.fit(x_train,y_train.to_numpy(),batch_size=10,epochs=10,verbose=1)

Epoch 1/10
700/700 [==============================] - 1s 922us/step - loss: 0.2690 - accuracy: 0.8871
Epoch 2/10
700/700 [==============================] - 1s 917us/step - loss: 0.2690 - accuracy: 0.8887
Epoch 3/10
700/700 [==============================] - 1s 975us/step - loss: 0.2697 - accuracy: 0.8881
Epoch 4/10
700/700 [==============================] - 1s 939us/step - loss: 0.2685 - accuracy: 0.8894
Epoch 5/10
700/700 [==============================] - 1s 924us/step - loss: 0.2680 - accuracy: 0.8884
Epoch 6/10
700/700 [==============================] - 1s 929us/step - loss: 0.2686 - accuracy: 0.8880
Epoch 7/10
700/700 [==============================] - 1s 923us/step - loss: 0.2686 - accuracy: 0.8867
Epoch 8/10
700/700 [==============================] - 1s 930us/step - loss: 0.2677 - accuracy: 0.8867
Epoch 9/10
700/700 [==============================] - 1s 963us/step - loss: 0.2677 - accuracy: 0.8891
Epoch 10/10
700/700 [==============================] - 1s 918us/step - loss: 0.266

In [84]:
y_pred = model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
y_pred

In [ ]:
y_test

In [88]:
# Testing accuracy
model.evaluate(x_test,y_test.to_numpy())

94/94 [==============================] - 0s 780us/step - loss: 0.4035 - accuracy: 0.8410


[0.40348002314567566, 0.8410000205039978]

In [89]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [90]:
confusion_matrix(y_test,y_pred)

array([[2266,  123],
       [ 354,  257]])

In [91]:
accuracy_score(y_pred,y_test)

0.841